# Experiment 03

### Training on Google Collab 

In [1]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

ModuleNotFoundError: No module named 'google'

In [ ]:
# Go to base directory
%cd /content/drive/MyDrive/University/Master-Thesis/Code

### Training on Local Machine

In [1]:
import os

original_working_directory_path = os.getcwd()
print("The original working directory is {0}".format(os.getcwd()))

def to_original_working_directory():
    os.chdir(original_working_directory_path)
    print(f"Changed to original working directory {original_working_directory_path}")

The original working directory is /Users/jankreischer/Library/Mobile Documents/com~apple~CloudDocs/Master-Thesis/Code/experiments/experiment_03


In [2]:
def to_root_working_directory():
    root_working_directory_path = os.path.join(original_working_directory_path, "../..")
    os.chdir(root_working_directory_path)
    print(f"Changed to root working directory {os.getcwd()}")

In [3]:
to_root_working_directory()

Changed to root working directory /Users/jankreischer/Library/Mobile Documents/com~apple~CloudDocs/Master-Thesis/Code


### 1. Dependencies

In [4]:
# Standard Dependencies
import sys
import os
import numpy as np
from time import time

In [5]:
# Local Dependencies
from experiments.experiment_03.environment import SensorEnvironment

/Users/jankreischer/opt/anaconda3/envs/FedRL-for-IT-Sec/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Global Dependencies
from src.data_provider import DataProvider
from src.agent import Agent
from src.custom_types import Behavior
from src.simulation_engine import SimulationEngine
from src.evaluation_utils import plot_learning, seed_random, evaluate_agent, \
    evaluate_agent_on_afterstates, get_pretrained_agent
from src.autoencoder_utils import get_pretrained_ae, evaluate_ae_on_afterstates, evaluate_ae_on_no_mtd_behavior, pretrain_ae_model, \
    evaluate_all_ds_as_ae_models, pretrain_all_ds_as_ae_models

### 2. Hyperparameters

In [7]:
# Hyperparams
GAMMA = 0.1
BATCH_SIZE = 100
BUFFER_SIZE = 500
MIN_REPLAY_SIZE = 100
EPSILON_START = 1.0
EPSILON_DEC = 1e-4
EPSILON_END = 0.01
TARGET_UPDATE_FREQ = 100
LEARNING_RATE = 1e-4
N_EPISODES = 10000
LOG_FREQ = 100
DIMS = 20
SAMPLES = 10

### 3. Training

In [8]:
experiment_base_dir = "experiments/experiment_03"
seed_random()
start = time()

# read in all preprocessed data for a simulated, supervised environment to sample from
# dtrain, dtest, atrain, atest = DataProvider.get_reduced_dimensions_with_pca_ds_as(DIMS,
#                                                                                   dir=f"{experiment_base_dir}/")
dtrain, dtest, atrain, atest, scaler = DataProvider.get_scaled_scaled_train_test_split_with_afterstates(
    scaling_minmax=True, scale_normal_only=True)

# get splits for RL & AD of normal data
ae_ds_train, dtrain_rl = DataProvider.split_ds_data_for_ae_and_rl(dtrain)
ae_ds_train = np.vstack((ae_ds_train, ae_ds_train)) # upsampling to have equal contribution with afterstates
dims = len(ae_ds_train[0, :-1])
ae_as_train, atrain_rl = DataProvider.split_as_data_for_ae_and_rl(atrain)

# MODEL trained on all ds and as normal data assumes the least -> MOST REALISTIC
pretrain_all_ds_as_ae_models(ae_ds_train, ae_as_train, num_std=2.5)
evaluate_all_ds_as_ae_models(dtrain_rl, atrain_rl, dims=dims, dir=dir)
exit(0)

# choose best of the previously trained autoencoders
dir = f"{experiment_base_dir}/trained_models/"
model_name = "ae_model_all_ds_as.pth"
path = dir + model_name
ae_interpreter = get_pretrained_ae(path=path, dims=dims)
print("Evaluating AE trained on all decision and afterstates normal")
print("---Evaluation on decision behaviors train---")
evaluate_ae_on_no_mtd_behavior(ae_interpreter, test_data=dtrain)
print("---Evaluation on afterstate behaviors train---")
evaluate_ae_on_afterstates(ae_interpreter, test_data=atrain)

# Reinforcement Learning
env = SensorEnvironment(decision_train_data=dtrain_rl,
                        after_train_data=atrain_rl, interpreter=ae_interpreter,
                        state_samples=SAMPLES)

agent = Agent(input_dims=env.observation_space_size, n_actions=len(env.actions), buffer_size=BUFFER_SIZE,
              batch_size=BATCH_SIZE, lr=LEARNING_RATE, gamma=GAMMA, epsilon=EPSILON_START, eps_end=EPSILON_END)

# initialize memory replay buffer (randomly)
SimulationEngine.init_replay_memory(agent=agent, env=env, min_size=MIN_REPLAY_SIZE)

# main training
episode_returns, eps_history = SimulationEngine.learn_agent_offline(agent=agent, env=env, num_episodes=N_EPISODES,
                                                                    t_update_freq=TARGET_UPDATE_FREQ)

end = time()
print("(Adapt!) Total training time: ", end - start)

# save pretrained agent for later (online) use
num = 0
agent.save_agent_state(num, f"{experiment_base_dir}")

x = [i + 1 for i in range(N_EPISODES)]
filename = f"{experiment_base_dir}/mtd_agent_p3_{SAMPLES}_samples.pdf"
plot_learning(x, episode_returns, eps_history, filename)

# check predictions with dqn from trained and stored agent
path = f"{experiment_base_dir}/trained_models/agent_{num}.pth"
pretrained_agent = get_pretrained_agent(path=path, input_dims=env.observation_space_size,
                                        n_actions=len(env.actions), buffer_size=BUFFER_SIZE)
evaluate_agent(agent=pretrained_agent, test_data=dtest)
evaluate_agent_on_afterstates(agent=pretrained_agent, test_data=atest)

size train: (2760, 46), size valid: (690, 46)
---Training AE---
AE threshold: 0.458658420230592
save model to: experiments/experiment_03/trained_models/ae_model_ds.pth
size train: (632, 46), size valid: (159, 46)
---Training AE---
AE threshold: 0.8644213560053753
save model to: experiments/experiment_03/trained_models/ae_model_rootkit_sanitizer.pth
size train: (704, 46), size valid: (177, 46)
---Training AE---
AE threshold: 0.9807522491911944
save model to: experiments/experiment_03/trained_models/ae_model_ransomware_directory_trap.pth
size train: (623, 46), size valid: (156, 46)
---Training AE---
AE threshold: 1.1035275906409003
save model to: experiments/experiment_03/trained_models/ae_model_ransomware_file_extension_hide.pth
size train: (610, 46), size valid: (153, 46)
---Training AE---
AE threshold: 1.5876529507731125
save model to: experiments/experiment_03/trained_models/ae_model_cnc_ip_shuffle.pth
all as data:  3214
size train: (2571, 46), size valid: (643, 46)
---Training AE---

TypeError: expected str, bytes or os.PathLike object, not builtin_function_or_method